<a href="https://colab.research.google.com/github/macgyver121/DADS7202_final_project/blob/main/prototype_evaluateV2%2Bmd_k2t_23_12_65.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check GPU

In [1]:
# List all NVIDIA GPUs as avaialble in this computer (or Colab's session)
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2920c881-f0c3-ed2d-009d-30ec56d4d31a)


In [ ]:
import sys
print( f"Python {sys.version}\n" )

import numpy as np
print( f"NumPy {np.__version__}\n" )

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print( f"TensorFlow {tf.__version__}" )
print( f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}" )

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print( f"TensorFlow detected { len(gpus) } GPU(s):" )
for i, gpu in enumerate(gpus):
  print( f".... GPU No. {i}: Name = {gpu.name} , Type = {gpu.device_type}" )

Python 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]

NumPy 1.21.6

TensorFlow 2.9.2
tf.keras.backend.image_data_format() = channels_last
TensorFlow detected 1 GPU(s):
.... GPU No. 0: Name = /physical_device:GPU:0 , Type = GPU


Install pytesseract

In [2]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install -q transformers datasets
!pip install -q pytorch-lightning wandb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (8,173 kB/s)
debconf: unable to initi

     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 452 kB 76.6 MB/s 
     |████████████████████████████████| 7.6 MB 63.9 MB/s 
     |████████████████████████████████| 182 kB 74.2 MB/s 
     |████████████████████████████████| 132 kB 81.4 MB/s 
     |████████████████████████████████| 212 kB 80.3 MB/s 
     |████████████████████████████████| 127 kB 79.7 MB/s 
     |████████████████████████████████| 800 kB 4.6 MB/s 
     |████████████████████████████████| 1.9 MB 73.7 MB/s 
     |████████████████████████████████| 125 kB 81.0 MB/s 
     |████████████████████████████████| 512 kB 84.1 MB/s 
     |████████████████████████████████| 182 kB 77.1 MB/s 
     |████████████████████████████████| 174 kB 79.7 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 173 kB 82.0 MB/s 
     |████████████████████████████████| 168 kB 82.7 MB/s 
     |████████████████████████████████| 168 kB 70.6 MB/s 
     |████████████

Reset runtime

In [ ]:
import os
os.kill(os.getpid(), 9)

Import library

In [34]:
import cv2
import pytesseract
import shutil
import os
import random
from PIL import Image

import re
import csv
import os

import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import re
import csv
import os
import pyarrow as pa
import torch
import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

Create Data from Chart2text data

In [3]:
List_key = []
# Open the CSV file for reading
for i in range(6763): 
  with open('/content/drive/MyDrive/DADS7202_final/Chart2text/data/'+str(i)+'.csv', 'r') as d:
 # Read the contents of the file into a list of rows
    rows = list(csv.reader(d))
# Concatenate the rows into a single string
    csv_string = '\n'.join(['\t'.join(row) for row in rows])
    csv_string = re.sub('\s+',' ',csv_string)
    List_key.append(csv_string)
len(List_key)

10

In [5]:
txt_key = []
for i in range(6763): 
 f = open('/content/drive/MyDrive/DADS7202_final/Chart2text/captions/'+str(i)+'.txt', 'r')
 content = f.read()
 content = re.sub('\s+',' ',content)
 txt_key.append(content)
len(txt_key)

10

In [6]:
print(List_key[0])

Quarter Number of users in millions Q4 '19 2498 Q3 '19 2449 Q2 '19 2414 Q1 '19 2375 Q4 '18 2320 Q3 '18 2271 Q2 '18 2234 Q1 '18 2196 Q4 '17 2129 Q3 '17 2072 Q2 '17 2006 Q1 '17 1936 Q4 '16 1860 Q3 '16 1788 Q2 '16 1712 Q1 '16 1654 Q4 '15 1591 Q3 '15 1545 Q2 '15 1490 Q1 '15 1441 Q4 '14 1393 Q3 '14 1350 Q2 '14 1317 Q1 '14 1276 Q4 '13 1228 Q3 '13 1189 Q2 '13 1155 Q1 '13 1110 Q4 '12 1056 Q3 '12 1007 Q2 '12 955 Q1 '12 901 Q4 '11 845 Q3 '11 800 Q2 '11 739 Q1 '11 680 Q4 '10 608 Q3 '10 550 Q2 '10 482 Q1 '10 431 Q4 '09 360 Q3 '09 305 Q2 '09 242 Q1 '09 197 Q3 '08 100


In [7]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
# Create a list of lists that represents the data
train_data = [List_key[:4059] ,txt_key[:4059]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[4059:5411] ,txt_key[4095:5411]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[5411:] ,txt_key[5411:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(dataset)  # Output: Dataset object with train_data
print(type(dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 6
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 2
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 2
})}
<class 'dict'>


In [9]:
from datasets import DatasetDict

dataset_dict = DatasetDict(dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 6
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 2
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 2
    })
})


In [10]:
print(dataset['train'][1])

{'key': "NBA player Number of points scored Kareem Abdul-Jabbar 38387 Karl Malone 36928 LeBron James* 33655 Kobe Bryant 33643 Michael Jordan 32292 Dirk Nowitzki 31560 Wilt Chamberlain 31419 Shaquille O'Neal 28596 Moses Malone 27409 Elvin Hayes 27313 Hakeem Olajuwon 26946 Oscar Robertson 26710 Dominique Wilkins 26668 Tim Duncan 26496 Paul Pierce 26397 John Havlicek 26395 Kevin Garnett 26071 Carmelo Anthony* 26067 Vince Carter* 25623 Alex English 25613 Reggie Miller 25279 Jerry West 25192 Patrick Ewing 24815 Ray Allen 24505 Allen Iverson 24368 Charles Barkley 23757 Robert Parish 23334 Adrian Dantley 23177 Dwyane Wade 23165 Elgin Baylor 23149 Kevin Durant* 22940", 'text': "How many points does LeBron James have ? In January 2020 , LeBron James passed Kobe Bryant on the NBA all-time scoring list , scoring 29 points to in a regular season game against the Philadelphia 76ers . If James continues to produce at about his career average , he has a shot to move up even further and pass Karl Malo

In [11]:
example = dataset_dict['train'][1]

print("Key:", example["key"])
print("Text:", example["text"])

Key: NBA player Number of points scored Kareem Abdul-Jabbar 38387 Karl Malone 36928 LeBron James* 33655 Kobe Bryant 33643 Michael Jordan 32292 Dirk Nowitzki 31560 Wilt Chamberlain 31419 Shaquille O'Neal 28596 Moses Malone 27409 Elvin Hayes 27313 Hakeem Olajuwon 26946 Oscar Robertson 26710 Dominique Wilkins 26668 Tim Duncan 26496 Paul Pierce 26397 John Havlicek 26395 Kevin Garnett 26071 Carmelo Anthony* 26067 Vince Carter* 25623 Alex English 25613 Reggie Miller 25279 Jerry West 25192 Patrick Ewing 24815 Ray Allen 24505 Allen Iverson 24368 Charles Barkley 23757 Robert Parish 23334 Adrian Dantley 23177 Dwyane Wade 23165 Elgin Baylor 23149 Kevin Durant* 22940
Text: How many points does LeBron James have ? In January 2020 , LeBron James passed Kobe Bryant on the NBA all-time scoring list , scoring 29 points to in a regular season game against the Philadelphia 76ers . If James continues to produce at about his career average , he has a shot to move up even further and pass Karl Malone with a

In [12]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

*************** เปลี่ยน tokenizer เป็นตาม huggingface เลือก model ****************

In [14]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")  ## edit here

prefix = "Text generation from keyword: "
max_input_length = 256
max_target_length = 128

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [15]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [17]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [18]:
!pip install -q pytorch-lightning wandb

******* ปรับ epoch กับ model เป็นตาม huggingface **********

In [19]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=10, warmup_steps=10):  ## edit here
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")  ## edit here
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [20]:
import wandb

wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [21]:
tokenizer.decode(batch['input_ids'][0])

"Text generation from keyword: Quarter Number of users in millions Q4 '19 2498 Q3 '19 2449 Q2 '19 2414 Q1 '19 2375 Q4 '18 2320 Q3 '18 2271 Q2 '18 2234 Q1 '18 2196 Q4 '17 2129 Q3 '17 2072 Q2 '17 2006 Q1 '17 1936 Q4 '16 1860 Q3 '16 1788 Q2 '16 1712 Q1 '16 1654 Q4 '15 1591 Q3 '15 1545 Q2 '15 1490 Q1 '15 1441 Q4 '14 1393 Q3 '14 1350 Q2 '14 1317 Q1 '14 1276 Q4 '13 1228 Q3 '13 1189 Q2 '13 1155 Q1 '13 1110 Q4 '12 1056 Q3 '12 1007 Q2 '12 955 Q1 '12 901 Q4 '11 845 Q3 '11 800 Q2 '11 739 Q1</s>"

In [22]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

"How many users does Facebook have? With almost 2.5 billion monthly active users as of the fourth quarter of 2019, Facebook is the biggest social network worldwide. In the third quarter of 2012, the number of active Facebook users surpassed one billion, making it the first social network ever to do so. Active users are those which have logged in to Facebook during the last 30 days. During the last reported quarter, the company stated that 2.9 billion people were using at least one of the company's core products ( Facebook, WhatsApp, Instagram, or Messenger ) each</s>"

In [23]:
model = CodeT5()

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

***** ถ้าใช้ k2t ให้ #logger=wandb_logger ด้วย **********

In [24]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

wandb_logger = WandbLogger(name='codet5-finetune-code-summarization-ruby-shuffle', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=3,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  logger=wandb_logger, 
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

wandb: Currently logged in as: srichaiperksiv (dads7202). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:30

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1595: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

************* ตั้ง path ที่จะ save model อันแรก ***********

In [25]:
save_directory = "/content/drive/MyDrive/model_test"
 # save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [26]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 2
})


In [28]:
test_example = dataset2['test'][0]
print("Key:", test_example['key'])

Key: Platform Number of active users in millions Facebook 2449 YouTube 2000 WhatsApp* 1600 Facebook Messenger* 1300 Weixin / WeChat 1151 Instagram* 1000 Douyin / Tik Tok 800 QQ 731 QZone 517 Sina Weibo 497 Reddit 430 Snapchat** 382 Twitter** 340 Pinterest 322 Kuaishou 316


In [ ]:
#from transformers import T5ForConditionalGeneration
#model = T5ForConditionalGeneration.from_pretrained(save_directory)

**************** AutoModelForSeq2SeqLM ของ แต่ละ model ให้ดูตาม huggingface ******************

In [30]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

Generate data from own model

In [31]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=120, max_length=150)
print("Generated :", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated : Facebook 2449 YouTube 2000 WhatsApp* 1600 Facebook Messenger* 1300 Weixin / WeChat 1151 Instagram* 1000 Douyin / Tik Tok 800 QQ 731 QZone 517 Sina Weibo 497 Reddit 430 Snapchat** 382 Twitter** 340 Pinterest 322 Kuaishou 316 Facebook 2449 YouTube 2000 WhatsApp* 1600 Facebook Messenger* 1300 Weixin / WeChat 1151 Instagram* 1000 Douyin / Tik Tok 800 Q..................................................................


Create list of predict data and ref data

In [ ]:
predict = []
refer = []
for i in range(1352):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=120, max_length=150)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
print(len(predict))
print(len(refer))

20
20


Evaluation

In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 4.4 MB/s 


In [ ]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.00031953827648916367, 'precisions': [0.1203281677301732, 0.025901942645698426, 0.01596244131455399, 0.010486177311725452], 'brevity_penalty': 0.011889643981689664, 'length_ratio': 0.1840912904849807, 'translation_length': 1097, 'reference_length': 5959}


In [ ]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(predict, refer, avg = True)
print(scores)

{'rouge-1': {'r': 0.02718855563573879, 'p': 0.3543644193578987, 'f': 0.04367451522714888}, 'rouge-2': {'r': 0.0026932797206510274, 'p': 0.024251423395445136, 'f': 0.004519768701117076}, 'rouge-l': {'r': 0.02632666937646645, 'p': 0.35046636838338596, 'f': 0.04226525984722125}}


**Ielt DATASET**

*********** แก้ path ************

In [35]:
List_key = []
for i in range(len(os.listdir('/content/drive/MyDrive/DADS7202_final/IELTS/table'))): 
  src = '/content/drive/MyDrive/DADS7202_final/IELTS/table/'+str(i)+'.png'
  img = cv2.imread(src)
  # Adding custom options
  custom_config = r'--psm 1'
  text = pytesseract.image_to_string(img, config=custom_config)
  #print(text)
  text_split = text.split('\n')
  #print(text_split)
  while '' in text_split:
    text_split.remove('')
  while ' ' in text_split:
    text_split.remove(' ')
  key = ' '.join(text_split[:-2])
  List_key.append(key)
len(List_key)

201

In [37]:
txt_key = []
for i in range(len(os.listdir('/content/drive/MyDrive/DADS7202_final/IELTS/captions'))): 
  f = open('/content/drive/MyDrive/DADS7202_final/IELTS/captions/'+str(i)+'.txt', 'r')
  content = f.read()
  content = re.sub('\s+',' ',content)
  txt_key.append(content)
len(txt_key)

201

In [38]:
import pyarrow as pa

# Create a list of lists that represents the data
train_data = [List_key[:121] ,txt_key[:121]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[121:161] ,txt_key[121:161]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[161:] ,txt_key[161:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
ielts_dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(ielts_dataset)  # Output: Dataset object with train_data
print(type(ielts_dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 121
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 40
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 40
})}
<class 'dict'>


In [39]:
print(ielts_dataset['train'][0])

{'key': 'Holiday leisure activities Number of hours |At the beach Reading books Chinese tourists 3 8 American tourists 4 5. Turkish tourists 5 4', 'text': 'The bar chart compares the amount of time each day that people from four different countries spent doing activities while on holiday in Greece in August 2019. The activities were going to the beach, visiting different attractions, and reading. Overall, the tourists spent the most time reading books while they spent the least time visiting places. The Chinese tourists spent the longest reading books with a total of eight hours per day. By contrast, the Brazilians spent the longest at the beach with a total of six hours. Both the American and the Turkish tourists spent the same number of hours every day visiting places with a total of three hours each. Similarly, the Turkish and Brazilian groups spent a total of four hours each reading books. The Chinese tourists spent twice the amount of time reading books as the Turkish and Brazilia

In [40]:
from datasets import DatasetDict

dataset_dict = DatasetDict(ielts_dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 121
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 40
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 40
    })
})


**Train ielts data**

In [41]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

*********** แก้ tokenizer ตาม model เรา **********

In [42]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

prefix = "Text generation from keyword: "
max_input_length = 256
max_target_length = 128

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [43]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [44]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [45]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [46]:
!pip install -q pytorch-lightning wandb

*********** แก้ AutoModelForSeq2SeqLM ตามแต่ละโมเดลตาม huggingface ********

********* แก้ epoch ***********

In [47]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=10, warmup_steps=100):  #edit here
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)  #edit here
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [ ]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [48]:
tokenizer.decode(batch['input_ids'][0])

'Text generation from keyword: World passenger car production, 2015 Percentage of car production 2015 Greater China 26% Japan/ Korea 16% Europe 25% South Asia 8% North America 19% South America 4%</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [49]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

"The pie chart gives the percentage of passenger cars manufactured by different regions of the world. For the purposes of this data set, the world is divided into seven regions. What stands out is that the regions of Greater China and Europe lead the way in terms of the manufacture of passenger cars, with 26% and 25% respectively. By contrast, very few cars are produced in the Middle East and Africa: just 2% originate from that area. Turning next to the Americas, a sharp contrast can be seen between the Northern and Southern parts of the continent. Whereas North America can boast 19% of the world's passenger</s>"

Create new model to train again

In [51]:
model = CodeT5()

In [52]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

wandb_logger = WandbLogger(name='codet5-finetune-code-summarization-ruby-shuffle', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=3,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  #logger=wandb_logger, 
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loops/utilities.py:94: 

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1595: PossibleUserWarning: The number of training batches (16) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

******* set new directory ที่จะsaveใน model อันใหม่หลัง train กับ ielts แล้ว ********

In [53]:
save_directory = "/content/drive/MyDrive/model_test2"
 # save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [54]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 40
})


In [55]:
test_example = dataset2['test'][0]
print("Key:", test_example['key'])

Key: % of population 65 andover 1980 2000 2030 Cannada 10% 15% 22% Germany 15% 20% 25%


******** แก้ AutoModelForSeq2SeqLM ตาม huggingface

In [56]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

In [57]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=120, max_length=150)
print("Generated :", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated : Cannada 10% 15% 22% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% Germany 15% 20% 25% of the population. Germany 15% 20% 20% 25% Germany 15% % of the population 65 and over.


Evaluation

In [58]:
predict = []
refer = []
for i in range(20):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=120, max_length=150)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

In [59]:
print(len(predict))
print(len(refer))

20
20


In [60]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 3.8 MB/s 


In [61]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.0065951380532341276, 'precisions': [0.18502202643171806, 0.03155555555555556, 0.0053811659192825115, 0.0013574660633484162], 'brevity_penalty': 0.4589313586402705, 'length_ratio': 0.5621594848935116, 'translation_length': 2270, 'reference_length': 4038}


In [62]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(predict, refer, avg = True)
print(scores)

{'rouge-1': {'r': 0.08153008008319398, 'p': 0.38098632807651167, 'f': 0.1324488764109736}, 'rouge-2': {'r': 0.008910027330838125, 'p': 0.04428553331307768, 'f': 0.014680179757025052}, 'rouge-l': {'r': 0.07593631972770801, 'p': 0.3614954836718742, 'f': 0.12381139764685649}}
